In [14]:
import pandas as pd
import subprocess
import os
import tempfile
from Bio import SeqIO # Using Biopython for robust FASTA parsing
import numpy as np # For numerical operations, especially distances

# Re-using functions from the previous script
def define_caspase_enzymes(caspase_rules):
    """
    Programmatically defines custom caspase enzymes in RPG if they don't exist,
    using the correct rule syntax. Handles "This name exist" gracefully.
    Returns a set of all caspase enzyme names that are now available in RPG.
    """
    print("Checking and defining caspase enzymes in RPG...")
    
    all_available_caspase_names = set()

    try:
        # Using '-l' to list all enzymes, '-n' for non-interactive output
        # Adding timeout to prevent hanging if RPG has issues
        result = subprocess.run(['rpg', '-l', '-n'], capture_output=True, text=True, check=True, timeout=10)
        defined_enzymes_output = result.stdout
        for line in defined_enzymes_output.splitlines():
            parts = line.strip().split(maxsplit=2) 
            if len(parts) >= 2 and parts[0].isdigit():
                all_available_caspase_names.add(parts[1].strip())
        print(f"Existing RPG enzymes found: {all_available_caspase_names}")
    except (subprocess.CalledProcessError, subprocess.TimeoutExpired) as e:
        print(f"Warning: Error or timeout listing RPG enzymes. Details: {e}")
        if isinstance(e, subprocess.CalledProcessError):
            print(f"Stderr: {e.stderr.strip()}")
        print("Could not retrieve existing enzyme names reliably. Proceeding by attempting definition for all.")
    except FileNotFoundError:
        print("Error: 'rpg' command not found. Please ensure RPG is installed and in your PATH.")
        return set()

    for caspase_name, cleavage_rule_rpg_syntax in caspase_rules.items():
        if caspase_name in all_available_caspase_names:
            print(f"Enzyme '{caspase_name}' already listed by RPG. Skipping definition.")
            continue

        print(f"Attempting to define new enzyme: '{caspase_name}' with rule '{cleavage_rule_rpg_syntax}'...")
        try:
            result = subprocess.run(
                ['rpg', '-a', '-n', '-y', caspase_name, '-x', cleavage_rule_rpg_syntax],
                capture_output=True, text=True, check=False, timeout=10
            )
            if result.returncode == 0:
                print(f"Successfully defined '{caspase_name}'.")
                all_available_caspase_names.add(caspase_name)
            elif "This name exist" in result.stderr:
                print(f"Enzyme '{caspase_name}' already exists (as reported by RPG's stderr). Skipping definition.")
                all_available_caspase_names.add(caspase_name)
            else:
                print(f"ERROR defining '{caspase_name}': {result.stderr.strip()}")
        except subprocess.TimeoutExpired:
             print(f"Timeout occurred while defining '{caspase_name}'.")
        except FileNotFoundError:
            print("Error: 'rpg' command not found during enzyme definition.")
            return set()

    return all_available_caspase_names

def run_rpg_digestion(input_fasta_path, caspase_name, output_csv_path):
    """
    Runs RPG as a subprocess for a single caspase and saves results to CSV.
    """
    try:
        subprocess.run(
            ['rpg', '-i', input_fasta_path, '-e', caspase_name, '-f', 'csv', '-n', '-o', output_csv_path, '-d', 'sequential'],
            check=True,
            capture_output=True,
            text=True,
            timeout=300 
        )
        return True
    except subprocess.CalledProcessError as e:
        print(f"Error running RPG for '{caspase_name}':")
        print(f"  Command: {' '.join(e.cmd)}")
        print(f"  Return Code: {e.returncode}")
        print(f"  Stdout: {e.stdout.strip()}")
        print(f"  Stderr: {e.stderr.strip()}")
        return False
    except subprocess.TimeoutExpired:
        print(f"Timeout occurred while running RPG for '{caspase_name}'.")
        return False
    except FileNotFoundError:
        print("Error: 'rpg' command not found. Please ensure RPG is installed and in your PATH.")
        return False

# New helper functions for the updated pipeline
def parse_fasta(fasta_file):
    """
    Parses a FASTA file and returns a dictionary of {ncbi_id: sequence}.
    Assumes header format is <ncbi_id>|<description>.
    """
    print(f"Parsing FASTA file: {fasta_file}")
    sequences = {}
    try:
        for record in SeqIO.parse(fasta_file, "fasta"):
            ncbi_id = record.id.split('|')[0] # Extract NCBI ID from header
            sequences[ncbi_id] = str(record.seq)
        print(f"Found {len(sequences)} sequences in FASTA.")
    except Exception as e:
        print(f"Error parsing FASTA file {fasta_file}: {e}")
    return sequences

def load_and_filter_excel(excel_file, sheet_name='RITA'):
    """
    Loads and filters the Excel sheet based on 'type' and 'sig' columns.
    """
    print(f"Loading Excel file: {excel_file}, sheet: {sheet_name}")
    try:
        df_rita = pd.read_excel(excel_file, sheet_name=sheet_name, engine="openpyxl")
        original_rows = len(df_rita)
        print(f"Original rows in '{sheet_name}': {original_rows}")

        df_filtered = df_rita[
            (df_rita['type'].isin(['VP', 'VT'])) &
            (df_rita['sig'] == 'Yes')
        ].copy() # Use .copy() to avoid SettingWithCopyWarning
        
        print(f"Filtered rows (type in [VP, VT] and sig=Yes): {len(df_filtered)}")
        return df_filtered
    except FileNotFoundError:
        print(f"Error: Excel file not found at {excel_file}")
        return pd.DataFrame()
    except Exception as e:
        print(f"Error loading or filtering Excel sheet '{sheet_name}' from {excel_file}: {e}")
        return pd.DataFrame()

def process_protein_cleavages(unique_proteins_fasta_path, caspase_motifs_rpg, currently_available_rpg_enzymes, temp_dir):
    """
    Runs RPG for all available caspases on a unique set of proteins
    and collects all cleavage sites.
    Returns a dictionary: {protein_id: {'caspases': [list_of_caspases], 'cleavage_sites': [sorted_list_of_1based_positions]}}
    """
    print("Processing unique proteins for caspase cleavage sites...")
    all_protein_cleavage_data = {}

    CLEAVAGE_POS_COL = 'Cleaving_pos' 
    HEADER_COL = 'Original_header'

    for caspase_name, motif_rule_rpg_syntax in caspase_motifs_rpg.items():
        if caspase_name not in currently_available_rpg_enzymes:
            print(f"Skipping RPG digestion for '{caspase_name}' as it's not defined or available.")
            continue
        
        # motif_str is not used here but could be for debugging, it's derived later
        temp_output_csv = os.path.join(temp_dir, f'rpg_output_{caspase_name}.csv')
        
        print(f"  Running RPG for '{caspase_name}'...") # Added this for more granularity
        if run_rpg_digestion(unique_proteins_fasta_path, caspase_name, temp_output_csv):
            try:
                df_rpg_output = pd.read_csv(temp_output_csv)
                
                if CLEAVAGE_POS_COL not in df_rpg_output.columns:
                    print(f"ERROR: Expected column '{CLEAVAGE_POS_COL}' not found in RPG output for '{caspase_name}'.")
                    print(f"Actual columns found: {df_rpg_output.columns.tolist()}")
                    continue
                if HEADER_COL not in df_rpg_output.columns:
                     print(f"ERROR: Expected header column '{HEADER_COL}' not found in RPG output for '{caspase_name}'.")
                     continue
                
                cleaved_peptides = df_rpg_output[
                    df_rpg_output[CLEAVAGE_POS_COL] > 0
                ]

                for index, row in cleaved_peptides.iterrows():
                    header_parts = str(row[HEADER_COL]).split('_')
                    original_protein_id = header_parts[0]

                    cleavage_point_1_based = int(row[CLEAVAGE_POS_COL])

                    if original_protein_id not in all_protein_cleavage_data:
                        all_protein_cleavage_data[original_protein_id] = {'caspases': set(), 'cleavage_sites': set()}
                    
                    all_protein_cleavage_data[original_protein_id]['caspases'].add(caspase_name)
                    all_protein_cleavage_data[original_protein_id]['cleavage_sites'].add(cleavage_point_1_based)

            except pd.errors.EmptyDataError:
                pass # No cleavage sites in this file, which is fine
            except Exception as e:
                print(f"Error reading or parsing RPG output for '{caspase_name}' from {temp_output_csv}: {e}")
    
    # Convert sets to sorted lists for consistent output
    for protein_id, data in all_protein_cleavage_data.items():
        data['caspases'] = sorted(list(data['caspases']))
        data['cleavage_sites'] = sorted(list(data['cleavage_sites']))
    
    print("Finished processing unique proteins for caspase cleavage sites.")
    return all_protein_cleavage_data

def get_cleavage_analysis(protein_seq, tile_seq, all_caspase_cleavage_data):
    """
    Analyzes tile cleavage and distance to nearest cleavage site.
    Cleavage at position P means the bond *after* residue P is broken.
    """
    protein_cleavable = False
    caspases_list = []
    full_protein_cleavage_sites_list = []
    
    if all_caspase_cleavage_data: # If there's any cleavage data for the protein
        protein_cleavable = True
        caspases_list = all_caspase_cleavage_data['caspases']
        full_protein_cleavage_sites_list = all_caspase_cleavage_data['cleavage_sites']
    
    tile_cut = False
    tile_in_x_snip = "N/A"
    distance_nearest_cleave = "N/A"

    if not protein_seq or not tile_seq:
        return protein_cleavable, ", ".join(caspases_list), full_protein_cleavage_sites_list, \
               tile_cut, tile_in_x_snip, distance_nearest_cleave

    # Find tile position in the full protein sequence (0-based)
    tile_start_0based = protein_seq.find(tile_seq)
    
    if tile_start_0based == -1:
        # Tile not found in the protein sequence
        tile_in_x_snip = "Tile not found in protein"
        return protein_cleavable, ", ".join(caspases_list), full_protein_cleavage_sites_list, \
               tile_cut, tile_in_x_snip, distance_nearest_cleave

    tile_end_0based = tile_start_0based + len(tile_seq)

    # Convert to 1-based for consistent comparison with cleavage sites
    tile_start_1based = tile_start_0based + 1
    tile_end_1based = tile_end_0based # This is the 1-based index of the *last residue* in the tile

    if full_protein_cleavage_sites_list:
        # Check if tile itself is cut
        for cp in full_protein_cleavage_sites_list:
            # A cleavage at 'cp' breaks the bond *after* residue 'cp'.
            # If cp is between the tile's start and its second-to-last residue, the tile is cut.
            # Condition: cleavage_point is after tile's start and before tile's end
            if tile_start_1based <= cp < tile_end_1based:
                tile_cut = True
                break
        
        # If tile not cut, find snippet and distance
        if not tile_cut:
            # Find the closest cleavage point before the tile's start
            cleavages_before_tile_start = [cp for cp in full_protein_cleavage_sites_list if cp < tile_start_1based]
            # The start of the snippet will be (nearest_cleavage_before + 1), or 1 if no cleavage before.
            snippet_start_1based = (max(cleavages_before_tile_start) + 1) if cleavages_before_tile_start else 1

            # Find the closest cleavage point after the tile's end
            cleavages_after_tile_end = [cp for cp in full_protein_cleavage_sites_list if cp >= tile_end_1based]
            # The end of the snippet will be (nearest_cleavage_after), or len(protein_seq) if no cleavage after.
            snippet_end_1based = min(cleavages_after_tile_end) if cleavages_after_tile_end else len(protein_seq)

            tile_in_x_snip = f"{snippet_start_1based}-{snippet_end_1based}"

            # Calculate distance to nearest cleavage outside the tile
            distances_outside = []
            if cleavages_before_tile_start:
                # Distance from tile's start to the closest cleavage point before it
                distances_outside.append(tile_start_1based - max(cleavages_before_tile_start) - 1)
            
            if cleavages_after_tile_end:
                # Distance from tile's end to the closest cleavage point after it
                distances_outside.append(min(cleavages_after_tile_end) - tile_end_1based)
            
            if distances_outside:
                distance_nearest_cleave = min(d for d in distances_outside if d >= 0) # Ensure distance is non-negative
            else:
                distance_nearest_cleave = "No cleavages outside tile"
    else:
        # No cleavages in the entire protein
        tile_in_x_snip = f"1-{len(protein_seq)}"
        distance_nearest_cleave = "No cleavages in protein"


    return protein_cleavable, ", ".join(caspases_list), full_protein_cleavage_sites_list, \
           tile_cut, tile_in_x_snip, distance_nearest_cleave

# Main pipeline function
def main_pipeline(fasta_file_path, excel_file_path, output_csv_path):
    # 1. Define Caspase Enzymes in RPG
    # Appended '_consens' to names for clarity and to ensure they are user-defined.
    caspase_motifs_rpg = {
        'Caspase-2_consens': '(V)(D)(V)(A)(D,)',
        'Caspase-3_consens': '(D)(E)(V)(D,)',
        'Caspase-6_consens': '(V)(E)(H)(D,)',
        'Caspase-7_consens': '(D)(E)(V)(D,)', 
        'Caspase-8_consens': '(I)(E)(T)(D,)',
        'Caspase-9_consens': '(L)(E)(H)(D,)',
        'Caspase-10_consens': '(A)(E)(V)(D,)'
    }
    currently_available_rpg_enzymes = define_caspase_enzymes(caspase_motifs_rpg)
    
    if not currently_available_rpg_enzymes:
        print("Critical Error: No caspase enzymes are available for digestion. Exiting.")
        return

    # 2. Parse FASTA file
    ncbi_fasta_sequences = parse_fasta(fasta_file_path)
    if not ncbi_fasta_sequences:
        print("Critical Error: No sequences parsed from FASTA file. Exiting.")
        return

    # 3. Load and Filter Excel Data
    df_filtered_excel = load_and_filter_excel(excel_file_path, sheet_name='RITA')
    if df_filtered_excel.empty:
        print("Critical Error: No data found or filtered from Excel sheet. Exiting.")
        return

    # 4. Merge Excel data with FASTA sequences
    # Add 'Protein_seq' column to the filtered DataFrame
    df_filtered_excel['Protein_seq'] = df_filtered_excel['NCBI_id'].map(ncbi_fasta_sequences)
    
    # Remove rows where NCBI_id did not match a FASTA sequence
    df_analysis = df_filtered_excel.dropna(subset=['Protein_seq']).copy()
    num_unmatched = len(df_filtered_excel) - len(df_analysis)
    if num_unmatched > 0:
        print(f"Warning: {num_unmatched} rows from Excel could not be matched to FASTA sequences by NCBI_id and were dropped.")
    
    if df_analysis.empty:
        print("Critical Error: No matched protein sequences for analysis. Exiting.")
        return

    # 5. Prepare unique proteins for RPG
    unique_protein_data = df_analysis[['NCBI_id', 'Protein_seq']].drop_duplicates().set_index('NCBI_id')
    
    temp_dir = tempfile.mkdtemp()
    unique_proteins_fasta_path = os.path.join(temp_dir, 'unique_proteins_for_rpg.fasta')

    with open(unique_proteins_fasta_path, 'w') as f:
        for ncbi_id, row in unique_protein_data.iterrows():
            f.write(f">{ncbi_id}\n{row['Protein_seq']}\n")
    print(f"Unique proteins ({len(unique_protein_data)}) written to temporary FASTA: {unique_proteins_fasta_path}")

    # 6. Run RPG for all caspases on unique proteins and collect results
    print("Initiating RPG digestion for all relevant caspases on unique proteins...")
    all_protein_cleavage_results = process_protein_cleavages(
        unique_proteins_fasta_path, caspase_motifs_rpg, currently_available_rpg_enzymes, temp_dir
    )
    print("RPG digestion and result collection complete.")

    # 7. Analyze each row of the filtered Excel data
    results_for_output = []
    print("Analyzing tiles and generating final output rows...")
    for index, row in df_analysis.iterrows():
        ncbi_id = row['NCBI_id']
        protein_seq = row['Protein_seq']
        
        # --- NEW CHANGE: Remove leading 'M' from Aminoacids (tile_seq) ---
        original_tile_seq = row['Aminoacids']
        tile_seq = original_tile_seq[1:] if original_tile_seq.startswith('M') else original_tile_seq
        # --- END NEW CHANGE ---

        cleavage_data_for_protein = all_protein_cleavage_results.get(ncbi_id, {})
        
        # Perform tile-specific cleavage analysis
        protein_cleavable, caspases_str, cleavage_sites_list, tile_cut, tile_in_x_snip, distance_nearest_cleave = \
            get_cleavage_analysis(protein_seq, tile_seq, cleavage_data_for_protein)

        # Prepare the output row
        output_row = row.to_dict() # Start with existing Excel columns
        output_row.update({
            'Protein_Cleavable': protein_cleavable,
            'Caspases': caspases_str,
            'Cleavag_sites': ", ".join(map(str, cleavage_sites_list)) if cleavage_sites_list else "None",
            'tile_cut': tile_cut,
            'tile_in_x_snip': tile_in_x_snip,
            'distance_nearest_cleave': distance_nearest_cleave,
            'Protein_seq': protein_seq
        })
        results_for_output.append(output_row)

    # 8. Create and Save Final Output DataFrame
    df_final_output = pd.DataFrame(results_for_output)
    
    # Select and reorder columns as per user request
    final_columns = [
        'tileID', 'log2FoldChange', 'padj', 'type', 'sig', 'Virus', 'NCBI_id', 'Uniprot_id', 'Gene_name', 'Aminoacids',
        'Protein_Cleavable', 'Caspases', 'Cleavag_sites', 'tile_cut', 'tile_in_x_snip', 'distance_nearest_cleave', 'Protein_seq'
    ]
    # Ensure all requested columns exist, fill missing with 'None' if any were somehow not added
    for col in final_columns:
        if col not in df_final_output.columns:
            df_final_output[col] = 'None' # Or pd.NA, depending on desired type

    df_final_output = df_final_output[final_columns]
    df_final_output = df_final_output.fillna('None') # Final pass for any NaNs

    try:
        df_final_output.to_csv(output_csv_path, index=False)
        print(f"\nPipeline complete. Results saved to {output_csv_path}")
    except Exception as e:
        print(f"Error saving final output CSV: {e}")
    finally:
        # Clean up temporary files and directory
        try:
            for f in os.listdir(temp_dir):
                os.remove(os.path.join(temp_dir, f))
            os.rmdir(temp_dir)
            print(f"Cleaned up temporary directory: {temp_dir}")
        except OSError as e:
            print(f"Error cleaning up temporary directory {temp_dir}: {e}")

if __name__ == "__main__":
    fasta_input = 'data/all_virus_proteins.fasta'
    excel_input = 'data/RITA_and_ABT_pos_selection_screens.xlsx'
    output_final_csv = 'results/RITA_virus_caspase_analysis.csv'
    
    main_pipeline(fasta_input, excel_input, output_final_csv)

Checking and defining caspase enzymes in RPG...
Existing RPG enzymes found: set()
Attempting to define new enzyme: 'Caspase-2_consens' with rule '(V)(D)(V)(A)(D,)'...
Enzyme 'Caspase-2_consens' already exists (as reported by RPG's stderr). Skipping definition.
Attempting to define new enzyme: 'Caspase-3_consens' with rule '(D)(E)(V)(D,)'...
Successfully defined 'Caspase-3_consens'.
Attempting to define new enzyme: 'Caspase-6_consens' with rule '(V)(E)(H)(D,)'...
Enzyme 'Caspase-6_consens' already exists (as reported by RPG's stderr). Skipping definition.
Attempting to define new enzyme: 'Caspase-7_consens' with rule '(D)(E)(V)(D,)'...
Successfully defined 'Caspase-7_consens'.
Attempting to define new enzyme: 'Caspase-8_consens' with rule '(I)(E)(T)(D,)'...
Enzyme 'Caspase-8_consens' already exists (as reported by RPG's stderr). Skipping definition.
Attempting to define new enzyme: 'Caspase-9_consens' with rule '(L)(E)(H)(D,)'...
Enzyme 'Caspase-9_consens' already exists (as reported by

# Run with full caspase library

In [7]:
import pandas as pd
import subprocess
import os
import tempfile
from Bio import SeqIO # Using Biopython for robust FASTA parsing
import numpy as np # For numerical operations, especially distances

# Re-using functions from the previous script
def get_all_rpg_enzymes():
    """
    Lists all enzymes available in RPG using 'rpg -l'.
    Returns a dictionary {enzyme_name: enzyme_id}.
    """
    print("Listing all available RPG enzymes...")
    all_enzymes = {}
    try:
        # We confirmed 'rpg -l' works directly and its output is what we want to capture.
        result = subprocess.run(['rpg', '-l'], capture_output=True, text=True, check=True, timeout=10)
        
        # You can add this line to see the raw output being parsed, for debugging
        # print("--- Raw RPG -l Output ---")
        # print(result.stdout)
        # print("--- End Raw RPG Output ---")

        if result.stderr:
            print(f"RPG stderr output during enzyme listing: {result.stderr.strip()}")

        for line in result.stdout.splitlines():
            # Example line: "1: Arg-C"
            parts = line.strip().split(':', 1) # Split only at the first colon
            if len(parts) >= 2:
                enzyme_id = parts[0].strip() # This will be "1"
                enzyme_name = parts[1].strip() # This will be "Arg-C"

                # No need for .replace('.', '') anymore, as we split on ':'
                # and .strip() removes whitespace.
                # Just ensure it's a digit now.
                if enzyme_id.isdigit():
                    all_enzymes[enzyme_name] = enzyme_id
        print(f"Found {len(all_enzymes)} enzymes in RPG.")
    except (subprocess.CalledProcessError, subprocess.TimeoutExpired) as e:
        print(f"Warning: Error or timeout listing RPG enzymes. Details: {e}")
        if isinstance(e, subprocess.CalledProcessError):
            print(f"Stderr: {e.stderr.strip()}")
        print("Could not retrieve existing enzyme names reliably.")
    except FileNotFoundError:
        print("Error: 'rpg' command not found. Please ensure RPG is installed and in your PATH.")
    return all_enzymes

def run_rpg_digestion(input_fasta_path, enzyme_name, output_csv_path): # Changed parameter name
    """
    Runs RPG as a subprocess for a single enzyme and saves results to CSV. # Updated docstring
    """
    try:
        subprocess.run(
            ['rpg', '-i', input_fasta_path, '-e', enzyme_name, '-f', 'csv', '-n', '-o', output_csv_path, '-d', 'sequential'], # Used new parameter name
            check=True,
            capture_output=True,
            text=True,
            timeout=300 
        )
        return True
    except subprocess.CalledProcessError as e:
        print(f"Error running RPG for '{enzyme_name}':") # Used new parameter name
        print(f"  Command: {' '.join(e.cmd)}")
        print(f"  Return Code: {e.returncode}")
        print(f"  Stdout: {e.stdout.strip()}")
        print(f"  Stderr: {e.stderr.strip()}")
        return False
    except subprocess.TimeoutExpired:
        print(f"Timeout occurred while running RPG for '{enzyme_name}'.") # Used new parameter name
        return False
    except FileNotFoundError:
        print("Error: 'rpg' command not found. Please ensure RPG is installed and in your PATH.")
        return False


# New helper functions for the updated pipeline
def parse_fasta(fasta_file):
    """
    Parses a FASTA file and returns a dictionary of {ncbi_id: sequence}.
    Assumes header format is <ncbi_id>|<description>.
    """
    print(f"Parsing FASTA file: {fasta_file}")
    sequences = {}
    try:
        for record in SeqIO.parse(fasta_file, "fasta"):
            ncbi_id = record.id.split('|')[0] # Extract NCBI ID from header
            sequences[ncbi_id] = str(record.seq)
        print(f"Found {len(sequences)} sequences in FASTA.")
    except Exception as e:
        print(f"Error parsing FASTA file {fasta_file}: {e}")
    return sequences

def load_and_filter_excel(excel_file, sheet_name='RITA'):
    """
    Loads and filters the Excel sheet based on 'type' and 'sig' columns.
    """
    print(f"Loading Excel file: {excel_file}, sheet: {sheet_name}")
    try:
        df_rita = pd.read_excel(excel_file, sheet_name=sheet_name, engine="openpyxl")
        original_rows = len(df_rita)
        print(f"Original rows in '{sheet_name}': {original_rows}")

        df_filtered = df_rita[
            (df_rita['type'].isin(['VP', 'VT'])) &
            (df_rita['sig'] == 'Yes')
        ].copy() # Use .copy() to avoid SettingWithCopyWarning
        
        print(f"Filtered rows (type in [VP, VT] and sig=Yes): {len(df_filtered)}")
        return df_filtered
    except FileNotFoundError:
        print(f"Error: Excel file not found at {excel_file}")
        return pd.DataFrame()
    except Exception as e:
        print(f"Error loading or filtering Excel sheet '{sheet_name}' from {excel_file}: {e}")
        return pd.DataFrame()

def process_protein_cleavages(unique_proteins_fasta_path, all_rpg_enzymes_dict, temp_dir):
    """
    Runs RPG for all available enzymes on a unique set of proteins
    and collects all cleavage sites, organized by protein and then by enzyme.
    Returns a dictionary: {protein_id: {enzyme_name: {'cleavage_sites': [sorted_list_of_1based_positions]}}}
    """
    print("Processing unique proteins for enzyme cleavage sites...")
    # New structure: protein_id -> enzyme_name -> {'cleavage_sites': set()}
    all_protein_cleavage_data = {}

    CLEAVAGE_POS_COL = 'Cleaving_pos' 
    HEADER_COL = 'Original_header'

    for enzyme_name in all_rpg_enzymes_dict.keys(): 
        temp_output_csv = os.path.join(temp_dir, f'rpg_output_{enzyme_name}.csv')
        
        print(f"  Running RPG for '{enzyme_name}'...")
        if run_rpg_digestion(unique_proteins_fasta_path, enzyme_name, temp_output_csv):
            try:
                df_rpg_output = pd.read_csv(temp_output_csv)
                
                if CLEAVAGE_POS_COL not in df_rpg_output.columns:
                    print(f"ERROR: Expected column '{CLEAVAGE_POS_COL}' not found in RPG output for '{enzyme_name}'.")
                    print(f"Actual columns found: {df_rpg_output.columns.tolist()}")
                    continue
                if HEADER_COL not in df_rpg_output.columns:
                     print(f"ERROR: Expected header column '{HEADER_COL}' not found in RPG output for '{enzyme_name}'.")
                     continue
                
                cleaved_peptides = df_rpg_output[
                    df_rpg_output[CLEAVAGE_POS_COL] > 0
                ]

                for index, row in cleaved_peptides.iterrows():
                    header_parts = str(row[HEADER_COL]).split('_')
                    original_protein_id = header_parts[0]

                    cleavage_point_1_based = int(row[CLEAVAGE_POS_COL])

                    if original_protein_id not in all_protein_cleavage_data:
                        all_protein_cleavage_data[original_protein_id] = {}
                    
                    if enzyme_name not in all_protein_cleavage_data[original_protein_id]:
                        all_protein_cleavage_data[original_protein_id][enzyme_name] = {'cleavage_sites': set()}
                    
                    all_protein_cleavage_data[original_protein_id][enzyme_name]['cleavage_sites'].add(cleavage_point_1_based)

            except pd.errors.EmptyDataError:
                pass # No cleavage sites in this file for this enzyme, which is fine
            except Exception as e:
                print(f"Error reading or parsing RPG output for '{enzyme_name}' from {temp_output_csv}: {e}")
    
    # Convert sets to sorted lists for consistent output
    for protein_id, enzyme_data in all_protein_cleavage_data.items():
        for enzyme_name, data in enzyme_data.items():
            data['cleavage_sites'] = sorted(list(data['cleavage_sites']))
    
    print("Finished processing unique proteins for enzyme cleavage sites.") 
    return all_protein_cleavage_data


def get_cleavage_analysis(protein_seq, tile_seq, enzyme_cleavage_sites_list): # Changed argument name
    """
    Analyzes tile cleavage and distance to nearest cleavage site for a SINGLE enzyme.
    Cleavage at position P means the bond *after* residue P is broken.
    """
    # protein_cleavable refers to whether THIS SPECIFIC enzyme has ANY cleavage site in the protein.
    protein_cleavable_by_enzyme = bool(enzyme_cleavage_sites_list)
    
    tile_cut = False
    tile_in_x_snip = "N/A"
    distance_nearest_cleave = "N/A"

    if not protein_seq or not tile_seq:
        # If no protein or tile sequence, cannot analyze
        return protein_cleavable_by_enzyme, enzyme_cleavage_sites_list, \
               tile_cut, tile_in_x_snip, distance_nearest_cleave

    # Find tile position in the full protein sequence (0-based)
    tile_start_0based = protein_seq.find(tile_seq)
    
    if tile_start_0based == -1:
        # Tile not found in the protein sequence
        tile_in_x_snip = "Tile not found in protein"
        return protein_cleavable_by_enzyme, enzyme_cleavage_sites_list, \
               tile_cut, tile_in_x_snip, distance_nearest_cleave

    tile_end_0based = tile_start_0based + len(tile_seq)

    # Convert to 1-based for consistent comparison with cleavage sites
    tile_start_1based = tile_start_0based + 1
    tile_end_1based = tile_end_0based # This is the 1-based index of the *last residue* in the tile

    if enzyme_cleavage_sites_list: # If this specific enzyme cleaves the protein at all
        # Check if tile itself is cut by this enzyme
        for cp in enzyme_cleavage_sites_list:
            if tile_start_1based <= cp < tile_end_1based:
                tile_cut = True
                break
        
        # If tile not cut, find snippet and distance (specific to this enzyme's cleavages)
        if not tile_cut:
            cleavages_before_tile_start = [cp for cp in enzyme_cleavage_sites_list if cp < tile_start_1based]
            snippet_start_1based = (max(cleavages_before_tile_start) + 1) if cleavages_before_tile_start else 1

            cleavages_after_tile_end = [cp for cp in enzyme_cleavage_sites_list if cp >= tile_end_1based]
            snippet_end_1based = min(cleavages_after_tile_end) if cleavages_after_tile_end else len(protein_seq)

            tile_in_x_snip = f"{snippet_start_1based}-{snippet_end_1based}"

            distances_outside = []
            if cleavages_before_tile_start:
                distances_outside.append(tile_start_1based - max(cleavages_before_tile_start) - 1)
            
            if cleavages_after_tile_end:
                distances_outside.append(min(cleavages_after_tile_end) - tile_end_1based)
            
            if distances_outside:
                distance_nearest_cleave = min(d for d in distances_outside if d >= 0) 
            else:
                distance_nearest_cleave = "No cleavages outside tile"
    else:
        # No cleavages by this specific enzyme in the entire protein
        tile_in_x_snip = f"1-{len(protein_seq)}"
        distance_nearest_cleave = "No cleavages by this enzyme"


    return protein_cleavable_by_enzyme, enzyme_cleavage_sites_list, \
           tile_cut, tile_in_x_snip, distance_nearest_cleave

# Main pipeline function
def main_pipeline(fasta_file_path, excel_file_path, output_csv_path):
    # 1. Get all available RPG enzymes (built-in and user-defined)
    all_rpg_enzymes_dict = get_all_rpg_enzymes()
    
    if not all_rpg_enzymes_dict:
        print("Critical Error: No RPG enzymes are available for digestion. Exiting.")
        return

    # 2. Parse FASTA file
    ncbi_fasta_sequences = parse_fasta(fasta_file_path)
    if not ncbi_fasta_sequences:
        print("Critical Error: No sequences parsed from FASTA file. Exiting.")
        return

    # 3. Load and Filter Excel Data
    df_filtered_excel = load_and_filter_excel(excel_file_path, sheet_name='RITA')
    if df_filtered_excel.empty:
        print("Critical Error: No data found or filtered from Excel sheet. Exiting.")
        return

    # 4. Merge Excel data with FASTA sequences
    df_filtered_excel['Protein_seq'] = df_filtered_excel['NCBI_id'].map(ncbi_fasta_sequences)
    
    df_analysis = df_filtered_excel.dropna(subset=['Protein_seq']).copy()
    num_unmatched = len(df_filtered_excel) - len(df_analysis)
    if num_unmatched > 0:
        print(f"Warning: {num_unmatched} rows from Excel could not be matched to FASTA sequences by NCBI_id and were dropped.")
    
    if df_analysis.empty:
        print("Critical Error: No matched protein sequences for analysis. Exiting.")
        return

    # 5. Prepare unique proteins for RPG
    unique_protein_data = df_analysis[['NCBI_id', 'Protein_seq']].drop_duplicates().set_index('NCBI_id')
    
    temp_dir = tempfile.mkdtemp()
    unique_proteins_fasta_path = os.path.join(temp_dir, 'unique_proteins_for_rpg.fasta')

    with open(unique_proteins_fasta_path, 'w') as f:
        for ncbi_id, row in unique_protein_data.iterrows():
            f.write(f">{ncbi_id}\n{row['Protein_seq']}\n")
    print(f"Unique proteins ({len(unique_protein_data)}) written to temporary FASTA: {unique_proteins_fasta_path}")

    # 6. Run RPG for all available enzymes on unique proteins and collect results
    print("Initiating RPG digestion for all available enzymes on unique proteins...")
    all_protein_cleavage_results = process_protein_cleavages(
        unique_proteins_fasta_path, all_rpg_enzymes_dict, temp_dir
    )
    print("RPG digestion and result collection complete.")

    # 7. Analyze each row of the filtered Excel data and generate enzyme-specific columns
    results_for_output = []
    print("Analyzing tiles and generating final output rows (per enzyme)...")
    
    sorted_enzyme_names = sorted(all_rpg_enzymes_dict.keys())

    for index, row in df_analysis.iterrows():
        ncbi_id = row['NCBI_id']
        protein_seq = row['Protein_seq']
        
        original_tile_seq = row['Aminoacids']
        tile_seq = original_tile_seq[1:] if original_tile_seq.startswith('M') else original_tile_seq

        output_row = row.to_dict() # Start with existing Excel columns

        # Initialize overall protein cleavable status
        overall_protein_cleavable = False 

        protein_cleavage_data_all_enzymes = all_protein_cleavage_results.get(ncbi_id, {})
        
        for enzyme_name in sorted_enzyme_names: 
            enzyme_specific_cleavage_data = protein_cleavage_data_all_enzymes.get(enzyme_name, {'cleavage_sites': []})
            enzyme_cleavage_sites_list = enzyme_specific_cleavage_data['cleavage_sites']

            protein_cleavable_by_enzyme, cleavage_sites_list_for_this_enzyme, tile_cut_by_enzyme, \
            tile_in_x_snip_by_enzyme, distance_nearest_cleave_by_enzyme = \
                get_cleavage_analysis(protein_seq, tile_seq, enzyme_cleavage_sites_list)

            # Update overall status if any enzyme cleaves the protein
            if protein_cleavable_by_enzyme:
                overall_protein_cleavable = True

            output_row.update({
                f'{enzyme_name}_protein_cleavable': protein_cleavable_by_enzyme,
                f'{enzyme_name}_cleavage_sites': ", ".join(map(str, cleavage_sites_list_for_this_enzyme)) if cleavage_sites_list_for_this_enzyme else "None",
                f'{enzyme_name}_tile_cut': tile_cut_by_enzyme,
                f'{enzyme_name}_tile_in_x_snip': tile_in_x_snip_by_enzyme,
                f'{enzyme_name}_distance_nearest_cleave': distance_nearest_cleave_by_enzyme
            })
        
        # Add the new summary column after iterating through all enzymes for the protein
        output_row['Overall_Protein_Cleavable'] = overall_protein_cleavable # New summary column

        results_for_output.append(output_row)

    # 8. Create and Save Final Output DataFrame
    df_final_output = pd.DataFrame(results_for_output)
    
    base_columns = [
        'tileID', 'log2FoldChange', 'padj', 'type', 'sig', 'Virus', 'NCBI_id', 'Uniprot_id', 'Gene_name', 'Aminoacids', 'Protein_seq',
        'Overall_Protein_Cleavable' # Add the new summary column here
    ]
    
    enzyme_specific_output_cols = []
    for enzyme_name in sorted_enzyme_names:
        enzyme_specific_output_cols.extend([
            f'{enzyme_name}_protein_cleavable',
            f'{enzyme_name}_cleavage_sites',
            f'{enzyme_name}_tile_cut',
            f'{enzyme_name}_tile_in_x_snip',
            f'{enzyme_name}_distance_nearest_cleave'
        ])

    final_columns = base_columns + enzyme_specific_output_cols

    for col in final_columns:
        if col not in df_final_output.columns:
            df_final_output[col] = 'None' 

    df_final_output = df_final_output[final_columns]
    df_final_output = df_final_output.fillna('None') 

    try:
        df_final_output.to_csv(output_csv_path, index=False)
        print(f"\nPipeline complete. Results saved to {output_csv_path}")
    except Exception as e:
        print(f"Error saving final output CSV: {e}")
    finally:
        try:
            for f in os.listdir(temp_dir):
                os.remove(os.path.join(temp_dir, f))
            os.rmdir(temp_dir)
            print(f"Cleaned up temporary directory: {temp_dir}")
        except OSError as e:
            print(f"Error cleaning up temporary directory {temp_dir}: {e}")

if __name__ == "__main__":
    fasta_input = 'data/all_virus_proteins.fasta'
    excel_input = 'data/RITA_and_ABT_pos_selection_screens.xlsx'
    output_final_csv = 'results/RITA_virus_cleavage_analysis.csv' 
    
    main_pipeline(fasta_input, excel_input, output_final_csv)

Listing all available RPG enzymes...
Found 54 enzymes in RPG.
Parsing FASTA file: data/all_virus_proteins.fasta
Found 1522 sequences in FASTA.
Loading Excel file: data/RITA_and_ABT_pos_selection_screens.xlsx, sheet: RITA
Original rows in 'RITA': 32721
Filtered rows (type in [VP, VT] and sig=Yes): 498
Unique proteins (324) written to temporary FASTA: /scratch/872575.1.mnemosyne-pub/tmpukublb_v/unique_proteins_for_rpg.fasta
Initiating RPG digestion for all available enzymes on unique proteins...
Processing unique proteins for enzyme cleavage sites...
  Running RPG for 'Arg-C'...
  Running RPG for 'Asp-N'...
  Running RPG for 'BNPS-Skatole'...
  Running RPG for 'Bromelain'...
  Running RPG for 'Caspase-1'...
  Running RPG for 'Caspase-2'...
  Running RPG for 'Caspase-3'...
  Running RPG for 'Caspase-4'...
  Running RPG for 'Caspase-5'...
  Running RPG for 'Caspase-6'...
  Running RPG for 'Caspase-7'...
  Running RPG for 'Caspase-8'...
  Running RPG for 'Caspase-9'...
  Running RPG for 'Ca